# 先端課題006 TPOTを用いた回帰予測

### 必要ライブラリをインストール

In [1]:
from tpot import TPOTRegressor
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

C:\Users\marur\Anaconda3\envs\apptask006\lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


### データ読み込み

In [28]:
train = pd.read_csv("./train.csv")
test =  pd.read_csv("./test.csv")
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

### tpotでは数値と欠損値しか扱えないので、数値以外の列をダミー変数化する関数

In [3]:
def change_objtype_toint(df):
    obj_df = df.select_dtypes(include="object")
    dummy_df = pd.get_dummies(obj_df)
#     もとのdfからobj列を排除
    drop_col_name = obj_df.columns.values
    df.drop(drop_col_name, axis=1, inplace=True)
# dummy化したDataframeとdropしたDataFrameをconcat
    new_df = pd.concat([dummy_df, df], axis=1)
    
    return new_df

### trainデータをtpotで扱えるように変更

In [4]:
train = change_objtype_toint(train)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 290 entries, MSZoning_C (all) to SalePrice
dtypes: float64(3), int64(35), uint8(252)
memory usage: 792.9 KB


### 検証用に説明変数と目的変数を分ける

In [5]:
X = train.drop("SalePrice", axis=1)
y = train["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    train_size=0.75,
                                                    test_size=0.25,
                                                    random_state=42)

### TPOTRegressorの設定

In [6]:
tpot = TPOTRegressor(scoring='r2',
                     generations=5,
                     population_size=25,
                     random_state=42,
                     verbosity=2,
                     n_jobs=-1)

### 学習

In [7]:
tpot.fit(X_train, y_train)

C:\Users\marur\Anaconda3\envs\apptask006\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


Imputing missing values in feature set
                                                                                                                       
Generation 1 - Current best internal CV score: 0.84437195421506
                                                                                                                       
Generation 2 - Current best internal CV score: 0.84437195421506
                                                                                                                       
Generation 3 - Current best internal CV score: 0.84437195421506
                                                                                                                       
Generation 4 - Current best internal CV score: 0.8508296046957303
                                                                                                                       
Generation 5 - Current best internal CV score: 0.8513022141157766
                                     

TPOTRegressor(generations=5, n_jobs=-1, population_size=25, random_state=42,
              scoring='r2', verbosity=2)

### 最終的な数値モデルを確認

In [8]:
tpot.fitted_pipeline_

Pipeline(steps=[('stackingestimator-1',
                 StackingEstimator(estimator=RidgeCV(alphas=array([ 0.1,  1. , 10. ])))),
                ('stackingestimator-2',
                 StackingEstimator(estimator=LassoLarsCV(normalize=False))),
                ('stackingestimator-3',
                 StackingEstimator(estimator=ElasticNetCV(l1_ratio=0.75,
                                                          random_state=42,
                                                          tol=0.01))),
                ('randomforestregressor',
                 RandomForestRegressor(max_features=0.4, min_samples_leaf=16,
                                       min_samples_split=14,
                                       random_state=42))])

### TPOTの設定を変更して結果がどう変化するかを確かめる

#### TPOTの引数
scoringパラメータでは、回帰問題のどの評価指標を用いてモデルを最適化するかを指定することができます。ここの例ではr2スコア（決定係数R2）を使います。指定できる評価指標は多数ありますので、TPOTのリファレンスページの（http://epistasislab.github.io/tpot/api/）Parameters>scoringの説明文をご覧ください。  
random_stateパラメータは乱数の種です。値を指定しておくと、毎回同じ学習結果が得られます。指定しなければ、毎回少し違った学習結果になります。  
generationsパラメータでは特徴量の最適化とパラメータチューニングの最適化を何回繰り返すかを指定できます。  
population_sizeではTPOTが使っている遺伝的アルゴリズム内のパラメータです。
verbosityパラメータは進捗状況を表示させるためのもので、プログレスバーを表示させる場合には2を設定します。  
n_jobsパラメータは並列演算を実施するためのもので、CPUのコア数を最大限に活用する場合には-1を設定します。  

In [9]:
# socreingを'neg_mean_squared_error'に変更
tpot = TPOTRegressor(scoring='neg_mean_squared_error',
                     generations=5,
                     population_size=25,
                     random_state=42,
                     verbosity=2,
                     n_jobs=-1)

tpot.fit(X_train, y_train)

Imputing missing values in feature set
                                                                                                                       
Generation 1 - Current best internal CV score: -925485658.6165469
                                                                                                                       
Generation 2 - Current best internal CV score: -886093621.0063703
                                                                                                                       
Generation 3 - Current best internal CV score: -870779121.3062241
                                                                                                                       
Generation 4 - Current best internal CV score: -870779121.3062241
                                                                                                                       
Generation 5 - Current best internal CV score: -870779121.3062241
                               

TPOTRegressor(generations=5, n_jobs=-1, population_size=25, random_state=42,
              scoring='neg_mean_squared_error', verbosity=2)

In [10]:
#generationを30に変更
tpot = TPOTRegressor(scoring='r2',
                     generations=30,
                     population_size=25,
                     random_state=42,
                     verbosity=2,
                     n_jobs=-1)

tpot.fit(X_train, y_train)

Imputing missing values in feature set
                                                                                                                       
Generation 1 - Current best internal CV score: 0.84437195421506
                                                                                                                       
Generation 2 - Current best internal CV score: 0.84437195421506
                                                                                                                       
Generation 3 - Current best internal CV score: 0.84437195421506
                                                                                                                       
Generation 4 - Current best internal CV score: 0.84437195421506
                                                                                                                       
Generation 5 - Current best internal CV score: 0.8525525037014706
                                       

TPOTRegressor(generations=30, n_jobs=-1, population_size=25, random_state=42,
              scoring='r2', verbosity=2)

In [13]:
#population_sizeを50に変更
tpot = TPOTRegressor(scoring='r2',
                     generations=5,
                     population_size=50,
                     random_state=42,
                     verbosity=2,
                     n_jobs=-1)

tpot.fit(X_train, y_train)

Imputing missing values in feature set
                                                                                                                       
Generation 1 - Current best internal CV score: 0.844516139906467
                                                                                                                       
Generation 2 - Current best internal CV score: 0.8522410870966104
                                                                                                                       
Generation 3 - Current best internal CV score: 0.8522410870966104
                                                                                                                       
Generation 4 - Current best internal CV score: 0.8522410870966104
                                                                                                                       
Generation 5 - Current best internal CV score: 0.8522410870966104
                                

TPOTRegressor(generations=5, n_jobs=-1, population_size=50, random_state=42,
              scoring='r2', verbosity=2)

### testデータで検証

In [29]:
test = change_objtype_toint(test)

### 共通する列を抜き出し

In [30]:
common_col = []
for test_col in test.columns.values:
    if test_col in X.columns.values:
        common_col.append(test_col)

In [31]:
test = test.loc[:,common_col]
X = X.loc[:,common_col]

### 学習

In [32]:
tpot = TPOTRegressor(scoring='r2',
                     generations=5,
                     population_size=25,
                     random_state=42,
                     verbosity=2,
                     n_jobs=-1)
tpot.fit(X, y)

Imputing missing values in feature set
                                                                                                                       
Generation 1 - Current best internal CV score: 0.8735846904604904
                                                                                                                       
Generation 2 - Current best internal CV score: 0.8746144298630144
                                                                                                                       
Generation 3 - Current best internal CV score: 0.8746144298630144
                                                                                                                       
Generation 4 - Current best internal CV score: 0.8746144298630144
                                                                                                                       
Generation 5 - Current best internal CV score: 0.8746144298630144
                               

TPOTRegressor(generations=5, n_jobs=-1, population_size=25, random_state=42,
              scoring='r2', verbosity=2)

In [33]:
y_pred = tpot.predict(test)
y_pred

Imputing missing values in feature set


array([112027.64720875, 153768.39889231, 174596.5852358 , ...,
       160725.89478057, 119419.25428719, 226664.9290364 ])

In [34]:
tpot.fitted_pipeline_

Pipeline(steps=[('stackingestimator-1',
                 StackingEstimator(estimator=SGDRegressor(alpha=0.001, eta0=1.0,
                                                          l1_ratio=0.0,
                                                          learning_rate='constant',
                                                          loss='squared_loss',
                                                          penalty='elasticnet',
                                                          power_t=100.0,
                                                          random_state=42))),
                ('stackingestimator-2',
                 StackingEstimator(estimator=LassoLarsCV(normalize=True))),
                ('randomforestregressor',
                 RandomForestRegressor(max_features=0.7500000000000001,
                                       min_samples_leaf=11, min_samples_split=9,
                                       random_state=42))])

### 分析レポート

TPOTRegressorのpopulation_size, generation, socreingを変更して評価結果の変化を確認した  
population_sizeが大きくなると評価は良くなったが、2epoch目からは評価結果が横ばいであった
generationは大きくすると評価結果はよくなった、ただepoch数が大きくなるほど評価精度が上がるまで時間を要する  
scoringを変更すると評価精度は落ちた

ハイパーパラメータチューニングをする必要があると感じた  

pycartよりコード量は多く、処理時間も長いが、これまでpandas,sklearnを使って学習してきたように使えるので、個人的には使いやすいと感じた  
pycartと並行して使うとよさそうである  